In [117]:
import pandas as pd 
import hashlib

In [118]:
file_path = "1/marketing_sample_for_walmart_com-walmart_com_product_review__20200701_20201231__5k_data.tsv.txt"

dataset = pd.read_csv(file_path , sep="\t" , encoding="utf-8" , low_memory = False)

In [119]:
dataset.head(3)

,Uniq Id,Crawl Timestamp,Dataset Origin,Product Id,Product Barcode,Product Company Type Source,Product Brand Source,Product Brand Normalised Source,Product Name Source,Match Rank,...,Product Currency,Product Available Inventory,Product Image Url,Product Model Number,Product Tags,Product Contents,Product Rating,Product Reviews Count,Bsr,Joining Key
0,1705736792d82aa2f2d3caf1c07c53f4,2020-09-24 03:21:12 +0000,NaN,2e17bf4acecdece67fc00f07ad62c910,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,NaN,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",NaN,NaN,NaN,NaN,81350af1be98d3753cf964709f0c766a
1,95a9fe6f4810fcfc7ff244fd06784f11,2020-10-30 14:04:08 +0000,NaN,076e5854a62dd283c253d6bae415af1f,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/9c8e42e4-13a5...,NaN,"Nice 'n Easy Permanent Color, 111 Natural Medi...",NaN,NaN,NaN,NaN,0353e63907dc0de0c734db4690300057
2,8d4d0330178d3ed181b15a4102b287f2,2020-08-06 05:51:47 +0000,NaN,8a4fe5d9c7a6ed26cc44d785a454b124,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/e3a601c2-6a2b...,NaN,Clairol Nice 'N Easy Permanent Color 7/106A Na...,NaN,4.5,29221.0,NaN,b6985c8e94815fbca2319dbb8bf228af


In [120]:
dataset.isnull().sum()

Uniq Id                               0
Crawl Timestamp                       0
Dataset Origin                     5000
Product Id                            0
Product Barcode                    5000
Product Company Type Source           0
Product Brand Source               4861
Product Brand Normalised Source    4861
Product Name Source                4861
Match Rank                         5000
Match Score                        5000
Match Type                         5000
Retailer                              0
Product Category                     10
Product Brand                        13
Product Name                          0
Product Price                        42
Sku                                5000
Upc                                5000
Product Url                           0
Market                                0
Product Description                1127
Product Currency                      0
Product Available Inventory           0
Product Image Url                     0


In [121]:
dataset.columns

Index(['Uniq Id', 'Crawl Timestamp', 'Dataset Origin', 'Product Id',
       'Product Barcode', 'Product Company Type Source',
       'Product Brand Source', 'Product Brand Normalised Source',
       'Product Name Source', 'Match Rank', 'Match Score', 'Match Type',
       'Retailer', 'Product Category', 'Product Brand', 'Product Name',
       'Product Price', 'Sku', 'Upc', 'Product Url', 'Market',
       'Product Description', 'Product Currency',
       'Product Available Inventory', 'Product Image Url',
       'Product Model Number', 'Product Tags', 'Product Contents',
       'Product Rating', 'Product Reviews Count', 'Bsr', 'Joining Key'],
      dtype='object')

In [122]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Uniq Id                          5000 non-null   object 
 1   Crawl Timestamp                  5000 non-null   object 
 2   Dataset Origin                   0 non-null      float64
 3   Product Id                       5000 non-null   object 
 4   Product Barcode                  0 non-null      float64
 5   Product Company Type Source      5000 non-null   object 
 6   Product Brand Source             139 non-null    object 
 7   Product Brand Normalised Source  139 non-null    object 
 8   Product Name Source              139 non-null    object 
 9   Match Rank                       0 non-null      float64
 10  Match Score                      0 non-null      float64
 11  Match Type                       0 non-null      float64
 12  Retailer            

## Dropping unwanted columns

In [123]:
def deletecols(df , columns):
    return df.drop(columns = columns ,axis = 1 , inplace = True)
    

deletecols(dataset , ["Product Barcode" , "Sku" , "Upc","Product Model Number" , "Bsr","Product Contents","Match Score","Match Type" ,"Match Rank",
                     "Product Name Source","Product Brand Normalised Source" , "Product Contents","Product Brand Source","Product Available Inventory"
                     ,"Product Url","Market", "Retailer"])

In [124]:
dataset.shape

(5000, 16)

In [125]:
def hash_int(df , columne_name):
    df[columne_name] = (df[columne_name].astype(str)
                       .apply(lambda x : int(hashlib.md5(x.encode()).hexdigest(),16))
                       .astype("category").cat.codes
                       .astype("int"))
    return df

In [126]:
dataset = hash_int(dataset ,"Uniq Id")
dataset = hash_int(dataset , "Product Id")
dataset = hash_int(dataset , "Joining Key")

####  "Crawl Timestamp " converting the object type  into proper DateTime format 

In [127]:
dataset["Crawl Timestamp"] =pd.to_datetime(dataset["Crawl Timestamp"] , format="mixed" , errors = "coerce") 

In [128]:
## now extract date

dataset["Crawl_date"] = dataset["Crawl Timestamp"].dt.date

## new extract time 

dataset["Crawl_time"] = dataset["Crawl Timestamp"].dt.time


In [129]:
## Deleting the existing column  and joining key and Dataset Origin , we dont need it 
dataset.drop(["Crawl Timestamp","Joining Key" ,"Dataset Origin","Product Company Type Source"] , axis = 1 , inplace = True)

In [130]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Uniq Id                5000 non-null   int32  
 1   Product Id             5000 non-null   int32  
 2   Product Category       4990 non-null   object 
 3   Product Brand          4987 non-null   object 
 4   Product Name           5000 non-null   object 
 5   Product Price          4958 non-null   float64
 6   Product Description    3873 non-null   object 
 7   Product Currency       5000 non-null   object 
 8   Product Image Url      5000 non-null   object 
 9   Product Tags           5000 non-null   object 
 10  Product Rating         2194 non-null   float64
 11  Product Reviews Count  3346 non-null   float64
 12  Crawl_date             5000 non-null   object 
 13  Crawl_time             5000 non-null   object 
dtypes: float64(3), int32(2), object(9)
memory usage: 507.9+ 

In [131]:
dataset.head(5)

,Uniq Id,Product Id,Product Category,Product Brand,Product Name,Product Price,Product Description,Product Currency,Product Image Url,Product Tags,Product Rating,Product Reviews Count,Crawl_date,Crawl_time
0,752,4489,Premium Beauty > Premium Makeup > Premium Nail...,OPI,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",8.95,NaN,USD,https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",NaN,NaN,2020-09-24,03:21:12
1,3830,3370,Beauty > Hair Care > Hair Color > Auburn Hair ...,Nice'n Easy,"Nice n Easy Permanent Color, 111 Natural Mediu...",29.86,Pack of 3 Pack of 3 for the UPC: 381519000201 ...,USD,https://i5.walmartimages.com/asr/9c8e42e4-13a5...,"Nice 'n Easy Permanent Color, 111 Natural Medi...",NaN,NaN,2020-10-30,14:04:08
2,3293,117,Beauty > Hair Care > Hair Color > Permanent Ha...,Clairol,Clairol Nice N Easy Permanent Color 7/106A Nat...,7.99,This Clairol Nice N Easy Permanent Color gives...,USD,https://i5.walmartimages.com/asr/e3a601c2-6a2b...,Clairol Nice 'N Easy Permanent Color 7/106A Na...,4.5,29221.0,2020-08-06,05:51:47
3,1314,2474,Beauty > Makeup > Lip,Kokie Cosmetics,"Kokie Professional Matte Lipstick, Hot Berry, ...",5.16,Calling all matte lip lovers! Indulge in our r...,USD,https://i5.walmartimages.com/asr/25b4b467-bc61...,"Kokie Professional Matte Lipstick, Hot Berry, ...",NaN,NaN,2020-07-15,11:22:04
4,1875,40,Seasonal > Stock Up Essentials > Personal Care...,Gillette,"Gillette TRAC II Plus Razor Blade Refills, Fit...",19.97,"In 1971, Gillette introduced the Trac II razor...",USD,https://i5.walmartimages.com/asr/1a2ebb06-cd01...,"Gillette TRAC II Plus Razor Blade Refills, Fit...",NaN,131.0,2020-11-26,12:27:20


### Creating the dataset summary 

In [132]:
def summary(datasets):
    
    summary = pd.DataFrame({"Column Name": datasets.columns,
                            "Data Type": datasets.dtypes.astype(str),
                            "Missing Values": datasets.isnull().sum(),
                            "duplicate_rows": dataset.duplicated().sum(),
                            "Unique Values": datasets.nunique()
                           }).reset_index(drop=True)
    
    rows, cols = datasets.shape
    print(f"\n Rows: {rows}, Columns: {cols}\n")
    
    return summary

In [133]:
summary(dataset)


 Rows: 5000, Columns: 14



,Column Name,Data Type,Missing Values,duplicate_rows,Unique Values
0,Uniq Id,int32,0,0,5000
1,Product Id,int32,0,0,4802
2,Product Category,object,10,0,988
3,Product Brand,object,13,0,1600
4,Product Name,object,0,0,4921
5,Product Price,float64,42,0,2261
6,Product Description,object,1127,0,3759
7,Product Currency,object,0,0,1
8,Product Image Url,object,0,0,4945
9,Product Tags,object,0,0,4921


### Handling the missing values 

In [134]:
## we are imputing missing values with the help of category , brand annf global feedback

In [135]:
dataset["rating_for_model"] = dataset["Product Rating"]

## we are imputing only for our model 
# For each category:
#Calculate the mean rating ignoring null values 
#Replace null values within that category with that mean

dataset["rating_for_model"] = dataset.groupby('Product Category')["rating_for_model"].transform(lambda x:x.fillna(x.mean()))

## This is the edge case if remaining any null value that  will fill with mean 

dataset["rating_for_model"].fillna(dataset["Product Rating"].mean() , inplace=True)

C:\Users\ayush\AppData\Local\Temp\ipykernel_12932\569454432.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["rating_for_model"].fillna(dataset["Product Rating"].mean() , inplace=True)


In [136]:
dataset.drop("Product Rating" , axis =1 , inplace = True)

In [137]:
dataset["Product Reviews Count"] = dataset.groupby('Product Category')["Product Reviews Count"].transform(lambda x:x.fillna(x.mean()))

#edge case

dataset['Product Reviews Count'].fillna(dataset['Product Reviews Count'].median(), inplace=True)

C:\Users\ayush\AppData\Local\Temp\ipykernel_12932\562792881.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['Product Reviews Count'].fillna(dataset['Product Reviews Count'].median(), inplace=True)


In [138]:
## mode for categorical data 
dataset["Product Category"].fillna(dataset["Product Category"].mode()[0] , inplace = True)

dataset["Product Brand"].fillna(dataset["Product Brand"].mode()[0] , inplace = True)


C:\Users\ayush\AppData\Local\Temp\ipykernel_12932\3328427759.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["Product Category"].fillna(dataset["Product Category"].mode()[0] , inplace = True)
C:\Users\ayush\AppData\Local\Temp\ipykernel_12932\3328427759.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [139]:
dataset["Product Price"] = pd.to_numeric(dataset["Product Price"] , errors = "coerce")

In [140]:
## for price column  , category wise filling 

dataset["Product Price"] = dataset.groupby("Product Category")["Product Price"].transform(lambda x:x.fillna(x.median()))

## this is just an edge case 
dataset["Product Price"].fillna(dataset["Product Price"].median() , inplace= True)

C:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\ayush\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\a

In [141]:
dataset.isnull().sum()

Uniq Id                     0
Product Id                  0
Product Category            0
Product Brand               0
Product Name                0
Product Price               0
Product Description      1127
Product Currency            0
Product Image Url           0
Product Tags                0
Product Reviews Count       0
Crawl_date                  0
Crawl_time                  0
rating_for_model            0
dtype: int64

### Standardizing Product Image URLs to JPEG Format in Dataset

In [142]:
old_img_column = "Product Image Url"

new_img_column = "Product_image_Url_jpeg"

def to_convert(url):

    if pd.isna(url):
        return url
    # here we are removing the query parameter
    
    url = url.split("?")[0]

    # replacing the url extension with jpeg

    return url.rsplit(".",1)[0]+".jpeg"

    

In [143]:
## creating a new column and applying the above function 

dataset[new_img_column] = dataset[old_img_column].apply(to_convert)
dataset.drop(old_img_column , axis= 1 , inplace = True)

## creating a new updated database 

dataset.to_csv("Processed_data.csv",index = False)